# Import Library

In [ ]:
import os
import zipfile
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Cek versi Tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)

# Download dataset

In [ ]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

# Ekstrak file zip

In [ ]:
local_zip = 'rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

# Membagi file ke folder 'train' (60%) dan 'val' (40%)

In [ ]:
base_dir = '/tmp/rockpaperscissors'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Membuat subfolder rock, paper, dan scissors di dalam train dan validation
for folder in ['rock', 'paper', 'scissors']:
    os.makedirs(os.path.join(train_dir, folder))
    os.makedirs(os.path.join(validation_dir, folder))

# Memindahkan file ke dalam train dan validation
for folder in ['rock', 'paper', 'scissors']:
    source_folder = os.path.join(base_dir, folder)
    files = os.listdir(source_folder)
    random.shuffle(files)

    # Hitung jumlah file yang akan menjadi validation set (40% dari total)
    validation_count = int(0.4 * len(files))

    # Hitung jumlah file yang akan menjadi train set (60% dari total)
    train_count = len(files) - validation_count

    for i in range(validation_count):
        source_file = os.path.join(source_folder, files[i])
        dest_folder = os.path.join(validation_dir, folder)
        shutil.move(source_file, os.path.join(dest_folder, files[i]))

    # Memindahkan sisa file ke dalam subfolder train yang sesuai
    for i in range(validation_count, len(files)):
        source_file = os.path.join(source_folder, files[i])
        dest_folder = os.path.join(train_dir, folder)
        shutil.move(source_file, os.path.join(dest_folder, files[i]))

# Sekarang, direktori validation_dir akan berisi 40% dari jumlah keseluruhan data gabungan dari train dan validation,
# dan 60% data akan berada di dalam subfolder train yang sesuai dengan nama subfolder sebelumnya.


# Menghitung jumlah isi pada tiap folder

In [ ]:
base_dir = '/tmp/rockpaperscissors'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Fungsi untuk menghitung jumlah file dalam suatu direktori dan subdirektori
def count_files_in_directory(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len(files)
    return count

# Menghitung jumlah file dalam train dan validation
train_file_count = count_files_in_directory(train_dir)
validation_file_count = count_files_in_directory(validation_dir)

print(f"Jumlah file dalam train: {train_file_count}")
print(f"Jumlah file dalam validation: {validation_file_count}")


# Augmentasi Gambar dengan ImageDataGenerator()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest',
    width_shift_range=0.2,  # Pergeseran horizontal
    height_shift_range=0.2,  # Pergeseran vertikal
    validation_split=0.2  # Menambahkan pembagian data validasi
)

test_datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.2)

# Menyiapkan data latih dan validasi dari kumpulan data gambar yang di-load dalam memori melalui fungsi flow()

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 3 kelas, gunakan class_mode = 'categorical' (multiclass)
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 3 kelas, gunakan class_mode = 'categorical' (multiclass)
        class_mode='categorical')


# Membangun Model CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    # tf.keras.layers.Dense(128, activation='relu'),  # Lapisan hidden dengan 128 unit
    tf.keras.layers.Dense(3, activation='softmax')  # Mengubah unit menjadi 3 dan aktivasi menjadi 'softmax'
])

# melihat summary model

In [ ]:
model.summary()

# Melakukan Compile Model dan menentukan loss serta optimizer

In [ ]:
# Compile model dengan 'AdamW' optimizer dan loss function 'categorical_crossentropy'
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.AdamW(),  # optimizer diganti dengan 'AdamW'
              metrics=['accuracy'])

# Training Data

In [ ]:
# latih model dengan model.fit
model.fit(
      train_generator,
      steps_per_epoch=50,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=30, # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

# Melakukan Uji Model dengan input Gambar

In [ ]:
from google.colab import files

# Upload gambar yang ingin diuji
uploaded = files.upload()

# Daftar kelas
class_names = ['rock', 'paper', 'scissors']

for fn in uploaded.keys():
    # Memuat gambar dan meresize ke ukuran yang sesuai dengan model Anda
    path = fn
    img = image.load_img(path, target_size=(150, 150))

    # Menampilkan gambar yang diuji
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # Melakukan prediksi dengan model Anda
    predictions = model.predict(x)

    # Mendapatkan kelas hasil prediksi
    predicted_class = class_names[np.argmax(predictions)]

    print(f'File: {fn}')
    print(f'Prediction: {predicted_class}')
